In [17]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn 

import pyntcloud
import pymesh

import numpy as np
import pandas as pd

import multidim

import glob

In [13]:
label_fname = '../../data/shapenet/val.csv'

In [15]:
label_df = pd.read_csv(label_fname)

In [18]:
fname_list = glob.glob('../../data/shapenet/val_normal/*')

In [20]:
label_df.head()

,id,synsetId,subSynsetId
0,21817,4004475,4004475
1,2287,4004475,4004475
2,20666,4004475,4004475
3,40407,4004475,4004475
4,37999,4004475,4004475


In [29]:
int(fname_list[0].split('/')[-1].split('.')[0])

8937

In [32]:
label_df = label_df.set_index(keys='id')

In [35]:
label_df.loc[int(fname_list[0].split('/')[-1].split('.')[0])]

synsetId       2958343
subSynsetId    2958343
Name: 8937, dtype: int64

In [21]:
fname_list

['../../data/shapenet/val_normal/008937.obj',
 '../../data/shapenet/val_normal/025222.obj',
 '../../data/shapenet/val_normal/004040.obj',
 '../../data/shapenet/val_normal/030724.obj',
 '../../data/shapenet/val_normal/038291.obj',
 '../../data/shapenet/val_normal/025229.obj',
 '../../data/shapenet/val_normal/025063.obj',
 '../../data/shapenet/val_normal/007459.obj',
 '../../data/shapenet/val_normal/012540.obj',
 '../../data/shapenet/val_normal/011804.obj',
 '../../data/shapenet/val_normal/022127.obj',
 '../../data/shapenet/val_normal/051246.obj',
 '../../data/shapenet/val_normal/043032.obj',
 '../../data/shapenet/val_normal/046699.obj',
 '../../data/shapenet/val_normal/050373.obj',
 '../../data/shapenet/val_normal/006816.obj',
 '../../data/shapenet/val_normal/045540.obj',
 '../../data/shapenet/val_normal/005915.obj',
 '../../data/shapenet/val_normal/028541.obj',
 '../../data/shapenet/val_normal/044732.obj',
 '../../data/shapenet/val_normal/007297.obj',
 '../../data/shapenet/val_normal/0

In [36]:
clouds_all = []
labels_all = []
for fname in fname_list:
    label = int(fname.split('/')[-1].split('.')[0])
    mesh = pymesh.load_mesh(fname)
    clouds_all.append(mesh.vertices)
    labels_all.append(label)

In [51]:
labels_all = []
for fname in fname_list:
    model_id = int(fname.split('/')[-1].split('.')[0])
    labels_all.append(label_df.loc[model_id]['synsetId'])

Training and Testing
----------
Let's split our samples into training and testing

In [53]:
from sklearn.model_selection import cross_val_score, train_test_split
clouds_train, clouds_test, labels_train, labels_test = train_test_split(clouds_all, labels_all, test_size=0.2,
        random_state=20160529)
print("Number of training sets: {}".format(len(clouds_train)))
print("Number of testing sets: {}".format(len(clouds_test)))

Number of training sets: 4106
Number of testing sets: 1027


Make the big pointcloud
----------------------

We now merge all of the training data into one big weighted, labelled PointCloud object

In [60]:
pc_train = multidim.PointCloud.from_multisample_multilabel(clouds_train, labels_train)
print(pc_train)
print(pc_train.label_info)

A SimplicialComplex with 34069925 points, 0 edges, and 0 faces.
         clouds   points  weight  int_index
2691156     325  7083879     1.0          0
2747177      28    58551     1.0          1
2773838       7     7932     1.0          2
2801938       8    21868     1.0          3
2808440      72   253936     1.0          4
2818832      16    45464     1.0          5
2828884     156   602912     1.0          6
2843684       6     4601     1.0          7
2871439      39   124184     1.0          8
2876657      36    87147     1.0          9
2880940      13    41403     1.0         10
2924116      73  1119314     1.0         11
2933112     105   314246     1.0         12
2942699      10    16857     1.0         13
2946921       8     4443     1.0         14
2954340       5     6961     1.0         15
2958343     279  9460348     1.0         16
2992529      70   211932     1.0         17
3001627     536  2109753     1.0         18
3046257      52   145762     1.0         19
3085013     

In [61]:
from multidim.models import CDER
cder = CDER(parsimonious=True)

In [62]:
cder.fit(pc_train)
print("This CDER model uses {} Gaussians.".format(len(cder.gaussians)))

MemoryError: 